In [ ]:
import random
import csv
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt


In [ ]:
um= pd.read_csv('csv\\utility_matrix_1.csv',index_col=0)


In [ ]:
def standardize(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row
def center(row):
    new_row=(row-row.mean())
    return new_row

In [ ]:
def k_nearest_neighbor(arr,k):
    n,m=arr.shape
    #get the indexes of the (n-k) least similar items to set to 0 
    for i in range(n):
        arr[i][arr[i].argmax()]=0
        indexes=arr[i].argsort()[:m-k].tolist()
        arr[i][indexes]=0
    return(arr)

In [ ]:
def predict(utility_matrix, q_sim_neighbors, u_sim_neighbors, u_sim_weight, q_sim_weight):
    utility_matrix_copy = utility_matrix.fillna(0)
    output = utility_matrix_copy.copy()
    
    # Get the columns of the utility matrix
    columns = utility_matrix.axes[1]
    
    # Create similarity matrices for users and querries using cosine similarity and standardizing the utility matrix
    q_sim_matrix = cosine_similarity(utility_matrix_copy.T.apply(standardize))
    u_sim_matrix = cosine_similarity(utility_matrix_copy.apply(standardize))
    
    # Apply k-nearest neighbors to the similarity matrices using the input parameters
    u_sim_matrix = np.round(k_nearest_neighbor(u_sim_matrix, u_sim_neighbors), 3)
    q_sim_matrix = np.round(k_nearest_neighbor(q_sim_matrix, q_sim_neighbors), 3)
    
    # Find all the locations in the utility matrix where there is a missing value
    scores_to_predict = np.array(np.where(utility_matrix.fillna(0)== 0)).T
    for i,j in scores_to_predict:
        # Calculate the user similarity score and query similarity score
        q_score = np.dot(utility_matrix_copy.iloc[i], q_sim_matrix[j])/np.dot(q_sim_matrix[j], utility_matrix.iloc[i].notna())
        u_score = np.dot(utility_matrix_copy[columns[j]], u_sim_matrix[i])/np.dot(u_sim_matrix[i], utility_matrix[columns[j]].notna())
        
        # Make predictions by taking a weighted sum of the user similarity score and query similarity score
        output.iloc[i,j] = round(u_sim_weight*u_score + q_sim_weight*q_score, 2)
    return output


In [ ]:
print('Masking 20 % of the data to test')
mask = np.ones((2000,2000))
mask[1600:,1600:]=0
data_to_test=np.array(um)
data_to_test[np.logical_not(mask)] = None
data_to_test_df=pd.DataFrame(data_to_test,columns= um.axes[1],index=um.axes[0])
print('Mask done')

In [ ]:
x=np.arange(0,1.1,0.1)
y=[]
for i in x:
    print('q_sim_weight:',i)
    pred=predict(data_to_test_df,q_sim_weight=i,q_sim_neighbors=500, u_sim_neighbors=200,u_sim_weight=1-i)
    pred1=pred[um.notna()]
    y_pred=np.array(pred1.fillna(0))[np.logical_not(mask)]
    y_truth=np.array(um.fillna(0))[np.logical_not(mask)]
    y.append(sqrt(mean_squared_error(y_pred=y_pred,y_true=y_truth)))
    print('RMSE:',sqrt(mean_squared_error(y_pred=y_pred,y_true=y_truth)))